# PyPort Advanced Operations and Utilities

This notebook demonstrates advanced operations and utility functions in the PyPort library.

## Setup

First, we need to install the PyPort library if it's not already installed:

In [ ]:
# Uncomment and run this cell if you need to install PyPort
# !pip install pyport

Next, we'll import the necessary modules and set up our API credentials:

In [ ]:
import os
import json
import uuid
import time
from typing import Dict, Any, List
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, HTML

# Import PyPort
from pyport import PortClient
from pyport.exceptions import PortApiError
from pyport.utils import (
    clear_blueprint,
    save_snapshot,
    restore_snapshot,
    list_snapshots,
    delete_snapshot
)

# Helper function to pretty-print JSON
def print_json(data: Dict[str, Any]) -> None:
    """Print JSON data in a readable format."""
    print(json.dumps(data, indent=2))

Set your Port API credentials. You can either set them as environment variables or directly in this notebook:

In [ ]:
# Option 1: Get credentials from environment variables
client_id = os.environ.get('PORT_CLIENT_ID')
client_secret = os.environ.get('PORT_CLIENT_SECRET')

# Option 2: Set credentials directly (not recommended for shared notebooks)
# client_id = "your-client-id"
# client_secret = "your-client-secret"

# Check if credentials are set
if not client_id or not client_secret:
    print("Warning: PORT_CLIENT_ID and PORT_CLIENT_SECRET environment variables are not set.")
    print("Please set these variables or provide them directly in the notebook.")
else:
    print("✅ API credentials are set!")

## Initializing the Client

Now, let's initialize the Port client:

In [ ]:
# Initialize the Port client
client = PortClient(
    client_id=client_id,
    client_secret=client_secret
)

print("✅ Client initialized successfully!")

## Creating Test Data

Let's create some test data that we'll use for our advanced operations:

In [ ]:
def create_test_blueprint_with_entities(client: PortClient) -> Dict[str, Any]:
    """Create a test blueprint with entities.
    
    Args:
        client: The Port client.
        
    Returns:
        A dictionary with the IDs of the created resources.
    """
    # Generate a unique identifier for the test blueprint
    blueprint_id = f"test-service-{uuid.uuid4().hex[:8]}"
    
    # Create the blueprint
    blueprint_data = {
        "identifier": blueprint_id,
        "title": "Test Service",
        "icon": "Microservice",
        "schema": {
            "properties": {
                "language": {
                    "type": "string",
                    "title": "Language",
                    "enum": ["Python", "JavaScript", "Java", "Go", "Ruby"]
                },
                "version": {
                    "type": "string",
                    "title": "Version"
                },
                "status": {
                    "type": "string",
                    "title": "Status",
                    "enum": ["Active", "Deprecated", "In Development"]
                }
            }
        }
    }
    
    client.blueprints.create_blueprint(blueprint_data)
    print(f"✅ Created blueprint '{blueprint_id}'")
    
    # Create some entities
    entity_ids = []
    for i in range(3):
        entity_data = {
            "identifier": f"entity-{i}-{uuid.uuid4().hex[:8]}",
            "title": f"Test Entity {i+1}",
            "properties": {
                "language": ["Python", "JavaScript", "Java"][i % 3],
                "version": f"{i+1}.0.0",
                "status": ["Active", "In Development", "Deprecated"][i % 3]
            }
        }
        
        entity = client.entities.create_entity(blueprint_id, entity_data)
        entity_ids.append(entity["identifier"])
        print(f"✅ Created entity '{entity['identifier']}'")
    
    return {
        "blueprint_id": blueprint_id,
        "entity_ids": entity_ids
    }

# Create test data
try:
    test_data = create_test_blueprint_with_entities(client)
    blueprint_id = test_data["blueprint_id"]
    entity_ids = test_data["entity_ids"]
    print(f"\n✅ Test data created successfully!")
    print(f"Blueprint ID: {blueprint_id}")
    print(f"Entity IDs: {', '.join(entity_ids)}")
except PortApiError as e:
    print(f"❌ Error creating test data: {e}")
    blueprint_id = None
    entity_ids = []

## Using Utility Functions

### 1. Saving a Snapshot

Let's save a snapshot of our Port data:

In [ ]:
try:
    # Generate a unique prefix for our snapshot
    snapshot_prefix = f"test-snapshot-{uuid.uuid4().hex[:8]}"
    
    # Save a snapshot
    snapshot = save_snapshot(
        client,
        prefix=snapshot_prefix,
        include_blueprints=True,
        include_entities=True,
        include_actions=True,
        include_scorecards=True
    )
    
    print("✅ Snapshot saved successfully!")
    print("\nSnapshot details:")
    display(HTML(f"<b>ID:</b> {snapshot['id']}<br>"
                 f"<b>Created At:</b> {snapshot['createdAt']}<br>"
                 f"<b>Status:</b> {snapshot['status']}"))
    
    # Store the snapshot ID for later use
    snapshot_id = snapshot["id"]
except PortApiError as e:
    print(f"❌ Error saving snapshot: {e}")
    snapshot_id = None

### 2. Listing Snapshots

Let's list all available snapshots:

In [ ]:
try:
    # List all snapshots
    snapshots = list_snapshots(client)
    print(f"✅ Retrieved {len(snapshots)} snapshots!")
    
    # Convert to DataFrame for better visualization
    if snapshots:
        snapshot_data = []
        for snap in snapshots:
            snapshot_data.append({
                "id": snap["id"],
                "created_at": snap["createdAt"],
                "status": snap["status"],
                "size": snap.get("size", "Unknown")
            })
        
        # Create and display DataFrame
        snapshot_df = pd.DataFrame(snapshot_data)
        display(snapshot_df)
    else:
        print("No snapshots found.")
except PortApiError as e:
    print(f"❌ Error listing snapshots: {e}")

### 3. Clearing a Blueprint

Let's clear all entities from our test blueprint:

In [ ]:
if blueprint_id:
    try:
        # Get the current number of entities
        entities_before = client.entities.get_entities(blueprint_id)
        print(f"Blueprint '{blueprint_id}' has {len(entities_before)} entities before clearing.")
        
        # Clear the blueprint
        result = clear_blueprint(client, blueprint_id)
        print(f"✅ Cleared {result['deleted_entities']} entities from blueprint '{blueprint_id}'!")
        
        # Verify the entities were deleted
        entities_after = client.entities.get_entities(blueprint_id)
        print(f"Blueprint '{blueprint_id}' has {len(entities_after)} entities after clearing.")
    except PortApiError as e:
        print(f"❌ Error clearing blueprint: {e}")
else:
    print("No blueprint available to clear.")

### 4. Restoring a Snapshot

Now, let's restore the snapshot we saved earlier:

In [ ]:
if snapshot_id:
    try:
        # Restore the snapshot
        restore_result = restore_snapshot(
            client,
            snapshot_id,
            include_blueprints=True,
            include_entities=True,
            include_actions=True,
            include_scorecards=True
        )
        
        print("✅ Snapshot restoration initiated!")
        print("\nRestore details:")
        display(HTML(f"<b>Status:</b> {restore_result['status']}<br>"
                     f"<b>Message:</b> {restore_result.get('message', 'N/A')}"))
        
        # Wait a moment for the restoration to complete
        print("\nWaiting for restoration to complete...")
        time.sleep(5)
        
        # Verify the entities were restored
        if blueprint_id:
            entities = client.entities.get_entities(blueprint_id)
            print(f"\nBlueprint '{blueprint_id}' now has {len(entities)} entities after restoration!")
            
            # Display the restored entities
            if entities:
                entity_data = []
                for entity in entities:
                    entity_data.append({
                        "identifier": entity["identifier"],
                        "title": entity["title"],
                        "language": entity["properties"].get("language", ""),
                        "status": entity["properties"].get("status", "")
                    })
                
                # Create and display DataFrame
                entity_df = pd.DataFrame(entity_data)
                display(entity_df)
    except PortApiError as e:
        print(f"❌ Error restoring snapshot: {e}")
else:
    print("No snapshot available to restore.")

### 5. Deleting a Snapshot

Finally, let's delete the snapshot we created:

In [ ]:
if snapshot_id:
    try:
        # Delete the snapshot
        delete_result = delete_snapshot(client, snapshot_id)
        print(f"✅ Snapshot '{snapshot_id}' deleted successfully!")
        
        # Verify the snapshot was deleted
        snapshots = list_snapshots(client)
        snapshot_ids = [snap["id"] for snap in snapshots]
        if snapshot_id in snapshot_ids:
            print("❌ Snapshot still exists!")
        else:
            print("✅ Verified that the snapshot was deleted.")
    except PortApiError as e:
        print(f"❌ Error deleting snapshot: {e}")
else:
    print("No snapshot available to delete.")

## Advanced Search Operations

Let's explore some advanced search operations:

In [ ]:
if blueprint_id:
    try:
        # Complex search query with logical operators
        search_query = {
            "blueprint": blueprint_id,
            "$or": [
                {
                    "properties": {
                        "language": "Python"
                    }
                },
                {
                    "$and": [
                        {
                            "properties": {
                                "status": "Active"
                            }
                        },
                        {
                            "properties": {
                                "language": {
                                    "$ne": "Java"
                                }
                            }
                        }
                    ]
                }
            ]
        }
        
        # Execute the search
        results = client.entities.search_entities(search_query)
        print(f"✅ Advanced search returned {len(results)} results!")
        
        # Display the search results
        if results:
            result_data = []
            for entity in results:
                result_data.append({
                    "identifier": entity["identifier"],
                    "title": entity["title"],
                    "language": entity["properties"].get("language", ""),
                    "status": entity["properties"].get("status", "")
                })
            
            # Create and display DataFrame
            result_df = pd.DataFrame(result_data)
            display(result_df)
            
            # Explain the search results
            print("\nSearch Query Explanation:")
            print("This search finds entities that either:")
            print("1. Have 'Python' as their language, OR")
            print("2. Have 'Active' status AND a language that is NOT 'Java'")
        else:
            print("No entities matched the search criteria.")
    except PortApiError as e:
        print(f"❌ Error executing advanced search: {e}")
else:
    print("No blueprint available for search operations.")

## Error Handling and Retry Configuration

Let's demonstrate error handling and retry configuration:

In [ ]:
from pyport.retry import RetryConfig, RetryStrategy

# Create a client with custom retry configuration
retry_client = PortClient(
    client_id=client_id,
    client_secret=client_secret,
    max_retries=5,                      # Maximum number of retry attempts
    retry_delay=0.5,                    # Initial delay between retries (in seconds)
    max_delay=10.0,                     # Maximum delay between retries (in seconds)
    retry_strategy=RetryStrategy.EXPONENTIAL,  # Use exponential backoff
    retry_jitter=True,                  # Add jitter to retry delays
    retry_status_codes={429, 500, 502, 503, 504}  # Status codes to retry on
)

print("✅ Client with custom retry configuration initialized!")
print("\nRetry Configuration:")
display(HTML(f"<b>Max Retries:</b> 5<br>"
             f"<b>Initial Delay:</b> 0.5 seconds<br>"
             f"<b>Maximum Delay:</b> 10.0 seconds<br>"
             f"<b>Retry Strategy:</b> Exponential Backoff<br>"
             f"<b>Jitter:</b> Enabled<br>"
             f"<b>Status Codes to Retry:</b> 429, 500, 502, 503, 504"))

# Demonstrate error handling
print("\nDemonstrating Error Handling:")
try:
    # Try to get a non-existent blueprint
    non_existent_id = f"non-existent-{uuid.uuid4().hex}"
    blueprint = client.blueprints.get_blueprint(non_existent_id)
    print("✅ Blueprint retrieved successfully (this should not happen)!")
except PortApiError as e:
    print(f"✅ Caught expected error: {e}")
    print(f"  Status code: {e.status_code}")
    print(f"  Endpoint: {e.endpoint}")
    print(f"  Method: {e.method}")

## Cleaning Up

Let's clean up by deleting the test blueprint and entities we created:

In [ ]:
if blueprint_id:
    try:
        # Delete the blueprint
        success = client.blueprints.delete_blueprint(blueprint_id)
        if success:
            print(f"✅ Blueprint '{blueprint_id}' deleted successfully!")
        else:
            print(f"❌ Failed to delete blueprint '{blueprint_id}'.")
    except PortApiError as e:
        print(f"❌ Error deleting blueprint: {e}")
else:
    print("No blueprint to delete.")

## Summary

In this notebook, we've demonstrated advanced operations and utility functions in the PyPort library:

1. Using utility functions:
   - Saving snapshots
   - Listing snapshots
   - Clearing blueprints
   - Restoring snapshots
   - Deleting snapshots

2. Advanced search operations with complex queries and logical operators

3. Error handling and retry configuration

These advanced features provide powerful capabilities for managing your Port resources and implementing robust workflows.